# Scraping Metacritic for User Reviews of _The Host_

In this notebook, I will work on scraping user reviews of _The Host_ from [Metacritic](https://www.metacritic.com/movie/the-host/user-reviews?sort-by=date). The goal is to get a list of user reviews in the format I want, so that I can run analysis on these reviews and compare the results with user reviews of _Parasite_ !

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
from nltk.corpus import stopwords
import random
import re
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from collections import Counter

In [2]:
characters_to_strip = '().[]!,"'

In [3]:
%run functions.ipynb

In [4]:
url = 'https://www.metacritic.com/movie/the-host/user-reviews?sort-by=date'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


In [5]:
resp = requests.get(url, headers=headers)

In [6]:
html = resp.text

In [7]:
doc = BeautifulSoup(html, 'html.parser')

In [8]:
reviews = doc.find_all('div', attrs={'class':"review"})

In [9]:
len(reviews)

100

In [10]:
reviews[10]

<div class="review pad_top1">
<div class="left fl">
<div class="metascore_w user large movie positive indiv">8</div>
</div>
<div class="right fl">
<div class="title pad_btm_half"><span class="author"><a href="/user/TheRimRider">TheRimRider</a></span><span class="date">Nov  5, 2015</span></div>
<div class="summary">
<div class="review_body">
<span>Fresh, exciting and not your typical cliched Hollywood rubbish. Loved it. Great effects, real tension and all the elements you want in a horror/action film. Those who dont get it should go back to crap like Godzilla.</span>
</div>
</div>
<div class="interactions pad_top1">
<span class="helpful_wrapper">
<div class="helpful" data-mcrefid="5651533" data-mcreftype="230">
<div class="thumbs">
<span class="text"><span class="yes_count">0</span> of <span class="total_count">0</span> users found this helpful</span><span class="thumb_up"><i aria-hidden="true" class="fa fa-thumbs-up"></i><span class="count">0</span></span><span class="thumb_down"><i ar

Writing a function to extract the review data I want: date that the review was posted, the text of the review, and the score that the reviewer gave to the movie:

In [11]:
def extract_review_data(review_div):
    
    review = {}
    
    review['date'] = review_div.find('span', class_='date').text.strip()
    review['text'] = review_div.find('div', class_='review_body').text.strip()
    review['score'] = review_div.find('div', class_='metascore_w').text.strip()
    
    return review


In [12]:
extract_review_data(reviews[10])

{'date': 'Nov  5, 2015',
 'score': '8',
 'text': 'Fresh, exciting and not your typical cliched Hollywood rubbish. Loved it. Great effects, real tension and all the elements you want in a horror/action film. Those who dont get it should go back to crap like Godzilla.'}

In [13]:
review_data = []
for review in reviews:
    review_data.append(extract_review_data(review))

In [14]:
len(review_data)

100

We have 100 user reviews of _The Host_ ! Let's take a look at the first several reviews:

In [15]:
review_data[:5]

[{'date': 'Sep 30, 2020',
  'score': '9',
  'text': 'What starts a fun B-movie creature feature quickly turns into a dark and deeply depressing movie.'},
 {'date': 'May  1, 2020',
  'score': '8',
  'text': 'Registicamente è un capolavoro. La fotografia è stupenda e gli attori sono superlativi. Si potrebbe scomodare la parola capolavoro, se non fosse che, guardandolo nel 2020, ci si accorge dei pessimi effetti speciali. Assolutamente un must seeRegisticamente è un capolavoro. La fotografia è stupenda e gli attori sono superlativi. Si potrebbe scomodare la parola capolavoro, se non fosse che, guardandolo nel 2020, ci si accorge dei pessimi effetti speciali. Assolutamente un must see per tutti gli appassionati di cinema. Le critiche sulla pochezza della trama o sui personaggi buffoneschi si perdono completamente nella cornice stupenda che offre questo film.… Expand'},
 {'date': 'Mar 20, 2020',
  'score': '8',
  'text': 'The Host (2007)Sinopsis:\n\nPersonal militar estadounidense arroja de

Hm, it looks like some of these are NOT in English, which is a problem because I want to carry out analysis on English reviews. We'll tackle this issue in the `lang_detect` notebook.

In [16]:
review_data[36:40]

[{'date': 'Jan 29, 2008',
  'score': '5',
  'text': 'Why do these foreign films have to get a 0 or a 10? The Host is a solid 5 or 6. The family are a sort of Korean version of the family in "Liitle Miss Sunshine". A likeable mess that you want to win in the end. The horror takes a big second Why do these foreign films have to get a 0 or a 10? The Host is a solid 5 or 6. The family are a sort of Korean version of the family in "Liitle Miss Sunshine". A likeable mess that you want to win in the end. The horror takes a big second place to the comedy. Overall it wasn\'t a disaster, but it was no masterpiece, either as a thriller or as a relationship piece. As I said, it was a solid 5 or 6/10. Rent it if there\'s nothing else calling to you at Blockbuster. I\'m still trying to work out why so many professional critics pronounced it a masterpiece. It was fine, but nothing worth catching a bus in the rain for.… Expand'},
 {'date': 'Jan 22, 2008',
  'score': '10',
  'text': 'Fun, action-filled

In [15]:
ls

functions.ipynb                 nexis_parasite_critic.ipynb
metacritic_host_user.ipynb      README.md
metacritic_parasite_user.ipynb  rotten_tomatoes_host.ipynb
nexis_host_critic.ipynb


Writing this list out as a new JSON file:

In [17]:
with open('../data/user_reviews/metacritic_host_user.json','w', encoding='UTF-8') as out:
    out.write(json.dumps(review_data))

In [17]:
metacritic_host_user = json.load(open('../data/user_reviews/metacritic_host_user.json'))

Loading in the file above, and then now looking at a few of the reviews:

In [19]:
review_data[0]

{'date': 'Sep 30, 2020',
 'score': '9',
 'text': 'What starts a fun B-movie creature feature quickly turns into a dark and deeply depressing movie.'}

In [20]:
review_data[10]

{'date': 'Nov  5, 2015',
 'score': '8',
 'text': 'Fresh, exciting and not your typical cliched Hollywood rubbish. Loved it. Great effects, real tension and all the elements you want in a horror/action film. Those who dont get it should go back to crap like Godzilla.'}

In [21]:
review_data[0]['date']

'Sep 30, 2020'

In [22]:
for review in review_data[0:10]:
    print(review['text'])

What starts a fun B-movie creature feature quickly turns into a dark and deeply depressing movie.
Registicamente è un capolavoro. La fotografia è stupenda e gli attori sono superlativi. Si potrebbe scomodare la parola capolavoro, se non fosse che, guardandolo nel 2020, ci si accorge dei pessimi effetti speciali. Assolutamente un must seeRegisticamente è un capolavoro. La fotografia è stupenda e gli attori sono superlativi. Si potrebbe scomodare la parola capolavoro, se non fosse che, guardandolo nel 2020, ci si accorge dei pessimi effetti speciali. Assolutamente un must see per tutti gli appassionati di cinema. Le critiche sulla pochezza della trama o sui personaggi buffoneschi si perdono completamente nella cornice stupenda che offre questo film.… Expand
The Host (2007)Sinopsis:

Personal militar estadounidense arroja deshechos químicos al río Han de Corea del Sur. Varios años más tarde, una criatura emerge de sus turbias aguas y ataca furiosamente a los residentes locales. Cuando laT

### Initial data exploration

I want to see what the negative reviews are saying, particularly whether there are any 0 scores:

In [23]:
zero_scores = []

for review in review_data:
    if int(review['score'])<1:
        zero_scores.append(review)

In [24]:
zero_scores

[{'date': 'Apr 29, 2014',
  'score': '0',
  'text': 'One of the worst films I have ever seen. Not believable or thrilling in the least. The acting is so bad that it nixes the possibility of any suspension of disbelief. The film has no consistent tone and the characters are utterly ridiculous.'},
 {'date': 'Aug 23, 2010',
  'score': '0',
  'text': 'That was not a horror movie, it was a terrible piece of a movie!\rEven if this was a parody of a horror movie it wasnt good.\rNo scary parts.... really.\rIt sucked, waste of time :('},
 {'date': 'Aug 20, 2009',
  'score': '0',
  'text': 'This was the worst movie I have seen in quite some time. Incredibly corny with random attempts to beat the audience about the head with slapstick and physical humor that falls well short of the mark. A CGI monster that is way overrated.This was the worst movie I have seen in quite some time. Incredibly corny with random attempts to beat the audience about the head with slapstick and physical humor that falls 

Wow! Pretty harsh reviews. Let's take a look at exploring how many negative vs. positive reviews there are, based on the score each gave:

In [25]:
# Separating reviews by score (neg: 0-5 and pos: 6-10)

neg_review = []
pos_review = []

for review in review_data:
    if int(review['score']) < 6:
        neg_review.append(review)
    if int(review['score']) > 5:
        pos_review.append(review)

In [26]:
neg_review[:5]

[{'date': 'Jan 12, 2017',
  'score': '5',
  'text': 'Les films coréens sont bizarres en général et en particulier celui-ci, ce "drôle" de film avec un gros monstre dedans... un "drôle" de film, car on ne sait jamais si c\'est de lard ou du cochon, une caricature à l\'insu de son ignorance, uneLes films coréens sont bizarres en général et en particulier celui-ci, ce "drôle" de film avec un gros monstre dedans... un "drôle" de film, car on ne sait jamais si c\'est de lard ou du cochon, une caricature à l\'insu de son ignorance, une franche comédie ou un drame larmoyant mélodramatique violoneux comme les Coréens savent (aussi) le faire.Bref, on ne sait pas sur quel pied danser et c\'est fort désagréable parce que du coup, on a bien l\'air con. Quand le monstre bouffe des gens pour les régurgiter ensuite (!) on se dit que c\'est pour rigoler (?) ; le coup de l\'agent jaune (en référence sans nul doute à l\'agent orange du Vietnam) est-ce du pur cynisme ou une simple pique à l\'égard des Am

In [27]:
pos_review[:5]

[{'date': 'Sep 30, 2020',
  'score': '9',
  'text': 'What starts a fun B-movie creature feature quickly turns into a dark and deeply depressing movie.'},
 {'date': 'May  1, 2020',
  'score': '8',
  'text': 'Registicamente è un capolavoro. La fotografia è stupenda e gli attori sono superlativi. Si potrebbe scomodare la parola capolavoro, se non fosse che, guardandolo nel 2020, ci si accorge dei pessimi effetti speciali. Assolutamente un must seeRegisticamente è un capolavoro. La fotografia è stupenda e gli attori sono superlativi. Si potrebbe scomodare la parola capolavoro, se non fosse che, guardandolo nel 2020, ci si accorge dei pessimi effetti speciali. Assolutamente un must see per tutti gli appassionati di cinema. Le critiche sulla pochezza della trama o sui personaggi buffoneschi si perdono completamente nella cornice stupenda che offre questo film.… Expand'},
 {'date': 'Mar 20, 2020',
  'score': '8',
  'text': 'The Host (2007)Sinopsis:\n\nPersonal militar estadounidense arroja de

In [28]:
len(pos_review)

52

In [29]:
len(neg_review)

48

Pretty divided. I thought there might be more positive reviews (personally I enjoyed this movie), but I guess that's not the case. At least not for the 100 reviews I have here!

Next I'll try splitting by year. I want to see if I can look at just reviews of _The Host_ posted between 2007 and 2009, since reviews of _Parasite_ will also be limited to two year timespan (because it was released in 2019).

In [31]:
# Looking at date format:
review_data[0]['date']

'Sep 30, 2020'

mmm dd, yyyy

In [32]:
# Making a new list with just the year values from the date,
# And also making a new key:value pair for year

years = []

for review in review_data:
    years.append(review['date'][8:12])
    review['year']=int(review['date'][8:12])

In [33]:
review_data[10]

{'date': 'Nov  5, 2015',
 'score': '8',
 'text': 'Fresh, exciting and not your typical cliched Hollywood rubbish. Loved it. Great effects, real tension and all the elements you want in a horror/action film. Those who dont get it should go back to crap like Godzilla.',
 'year': 2015}

In [34]:
# Making a frequency list of years
years_freq = Counter(years)

In [35]:
# Most frequent years for reviews
years_freq.most_common()

[('2007', 57),
 ('2008', 12),
 ('2012', 5),
 ('2020', 4),
 ('2009', 4),
 ('2017', 3),
 ('2014', 3),
 ('2011', 3),
 ('2010', 3),
 ('2016', 2),
 ('2013', 2),
 ('2018', 1),
 ('2015', 1)]

In [36]:
type(review_data[0]['year'])

int

In [37]:
review_data[0]

{'date': 'Sep 30, 2020',
 'score': '9',
 'text': 'What starts a fun B-movie creature feature quickly turns into a dark and deeply depressing movie.',
 'year': 2020}

Filtering by year, since I only want user reviews that were posted between 2007 and 2009:

In [38]:
# Creating a new list with reviews posted earlier than 2010:

metacritic_host_user = []

for review in review_data:
    if review['year']<2010:
        rev_dict = {
            'date' : review['date'],
            'score' : review['score'],
            'text' : review['text']
        }
        metacritic_host_user.append(rev_dict)

To find how many of these user reviews were posted between 2007 and 2009, let's look at the length of our list:

In [39]:
len(metacritic_host_user)

73

In [41]:
# Looking at a sample user review in our new list:

metacritic_host_user[20]

{'date': 'Nov 27, 2007',
 'score': '9',
 'text': 'Excellent, it has many different elements, its a rich movie in acting, argument and production, funny and sad at the same time, I have a really good time watching this movie and will see it again.'}

So now we only have reviews posted between 2007 and 2009, and that ended up being 73 reviews. 

Now we want to save this new list out as a json file, so that we can work specifically with user reviews of _The Host_ from Metacritic that were posted within two years of the movie's release!

In [42]:
with open('../data/user_reviews/metacritic_host_user.json','w', encoding='UTF-8') as out:
    out.write(json.dumps(metacritic_host_user))

Let's try to see if we can also get critic reviews:

### Getting critic reviews from metacritic for _The Host_

In [5]:
critic_url = 'https://www.metacritic.com/movie/the-host/critic-reviews'

In [6]:
resp = requests.get(critic_url, headers=headers)

In [7]:
html = resp.text

In [8]:
doc = BeautifulSoup(html, 'html.parser')

In [9]:
review_divs = doc.find_all('div', attrs={'class':"review"})

In [10]:
len(review_divs)

36

There's only 36, which makes sense because I would assume there would be less critic reviews than user ones. 

In [11]:
review_divs[0]

<div class="review pad_top1 pad_btm1">
<div class="left fl">
<div class="metascore_w large movie positive indiv perfect">100</div>
</div>
<div class="right fl">
<div class="title pad_btm_half"><span class="source"><a href="/publication/chicago-reader?filter=movies"><img alt="Chicago Reader" class="pub-img" src="https://static.metacritic.com/images/publications/1567121885_901_633972424.png" title="Chicago Reader"/></a></span><span class="author"><a href="/critic/jr-jones?filter=movies">J.R. Jones</a></span></div>
<div class="summary">
<a class="no_hover" href="http://onfilm.chicagoreader.com/movies/critic.html#31160" rel="noopener" target="_blank">
                                                The mix of dark humor, creeping suspense, and a sort of apocalyptic tenderness makes this the best horror flick in years.
                                    </a>
<a class="read_full" href="http://onfilm.chicagoreader.com/movies/critic.html#31160" rel="noopener" target="_blank">Read full review<

Writing a function to get the information I want from the site:

In [12]:
def extract_criticreview_data(review_div):
    
    review = {}
    
    review['rating'] = review_div.find('div', class_='metascore_w').text.strip()
    review['text'] = review_div.find('div', class_='summary').text.strip()
    review['review_url'] = review_div.find('a', class_='read_full').attrs['href']
    try:
        review['source'] = review_div.find('span', class_='source').find('img').attrs['alt']
    except:
        review['source'] = review_div.find('span', class_='source').text
    review['critic'] = review_div.find('span', class_='author').text
    
    return review
    

In [13]:
extract_criticreview_data(review_divs[10])

{'critic': 'Manohla Dargis',
 'rating': '90',
 'review_url': 'http://movies2.nytimes.com/2007/03/09/movies/09host.html?ref=movies',
 'source': 'The New York Times',
 'text': 'The Host is a cautionary environmental tale about the domination of nature and the costs of human folly, and it may send chills up your spine. But only one will tickle your fancy and make you cry encore, not just uncle.\n                                    \nRead full review'}

In [14]:
criticreview_data=[]
for review in review_divs:
      try:
            criticreview_data.append(extract_criticreview_data(review))
      except:
            print('Error trying to extract review data')
            print(review)
            print()

Error trying to extract review data
<div class="ad_unit review pad_top1 pad_btm1" id="native_top">
<script type="text/javascript">
            
                            pushToDisplay('native_top', null, 'top', false, 'NmEGjUCMQTmJR3rtbiq2vRTC'), false;
                    </script>
</div>



In [15]:
len(criticreview_data)

35

It looks like there was one review on which this didn't work, so our list of critic reviews is only 35. 

We can try saving this out as a new JSON file:

In [24]:
with open('../data/critic_reviews/metacritic_host_critic.json','w', encoding='UTF-8') as out:
    out.write(json.dumps(criticreview_data, indent=4))

# adding the `indent=4` param to json.dumps will produce multiline output with indents that make
# it easier to read and edit the JSON file by hand

In [25]:
critic_review_data = json.load(open('../data/critic_reviews/metacritic_host_critic.json'))

Some of the reviews don't have the full text, so let's see how many of those there are:

In [26]:
missing_full_review=[r for r in critic_review_data if r['text'].endswith('Read full review')]

In [27]:
len(missing_full_review)

35

In [28]:
len(critic_review_data)

35

Unfortunately it looks like all of our reviews are missing full text. Again, I'm not sure if I will be able to carry out analysis on these because I hit paywalls or archived web pages when I try to retrieve the full text manually. 

In [29]:
missing_full_review

[{'critic': 'J.R. Jones',
  'rating': '100',
  'review_url': 'http://onfilm.chicagoreader.com/movies/critic.html#31160',
  'source': 'Chicago Reader',
  'text': 'The mix of dark humor, creeping suspense, and a sort of apocalyptic tenderness makes this the best horror flick in years.\n                                    \nRead full review'},
 {'critic': 'Peter Hartlaub',
  'rating': '100',
  'review_url': 'http://sfgate.com/cgi-bin/article.cgi?f=/c/a/2007/03/09/DDGMPOH6S31.DTL&type=movies',
  'source': 'San Francisco Chronicle',
  'text': 'A great piece of filmmaking and a legitimate science-fiction/horror classic.\n                                    \nRead full review'},
 {'critic': 'Felix Vasquez, Jr.',
  'rating': '100',
  'review_url': 'http://www.filmthreat.com/index.php?section=reviews&Id=9479',
  'source': 'Film Threat',
  'text': "Joon-Ho's epic is a masterpiece of monster cinema that's intelligent, innovative, and reaches down to the basic core of family unity to propel its st

Trying to retrieve some of the full text reviews manually, and formatting for JSON file:

In [33]:
'''
The Host is a strange little movie: part creature feature, part social commentary, and part slapstick comedy. The problem with the film is that the sum isn't greater than the parts and the pieces don't fuse in a way that's consistently pleasing or cinematically satisfying. While the monster movie aspects of the film represent great fun, The Host's political subplot is overlong, uninteresting, and obvious. The comedy, which surfaces in seemingly random, broad bursts, is more often bizarre than funny (people taking pratfalls are the most common), and the ending is a downer.

The premise is simple enough: Chemicals dumped into the Han River from a U.S. military base create a mutant creature that grows alarmingly large. One day, the thing emerges from the water and starts eating passers-by. Hyun-seo (Ko Ah-sung), the pre-teen daughter of lazy Seoul snack vendor Gang-du (Song Kang-ho), is abducted by the monster and taken to its lair for future consumption. Gang-du mounts a rescue operation, aided by his father, Hie-bong (Byeon Hie-bong); his sister, archery medalist Nam-joo (Bae Du-na); and his brother, alcoholic Nam-il (Park Hae-il). However, the government is concerned about a SARS-like epidemic and Gang-du and his family end up in quarantine while the monster runs rampant and shows interest in a Hyun-seo snack.

The creature feature aspects of The Host are enjoyable, even if the characters never achieve more than a semblance of two-dimensionality. This is monster movie making in the great Japanese tradition, although with better special effects. Like Godzilla, the monstrosity in The Host has been awakened and mutated by modern technology. This is what happens when you pollute the water! The film's best scenes are those involving Hyun-seo trapped in the creature's lair. Her cat-and-mouse game with it generates a good deal of tension; it's annoying when the movie cuts from her scenes to those involving pointless secondary characters involved in the political/virus subplot. Indeed, most of the scenes involving the monster are riveting, especially its high energy first appearance and the tense final standoff. Every time the movie becomes somnambulant (a lapse that occurs too often), the creature makes an appearance to give a jolt of energy to the story and the audience.

Fans of B-grade monster movies will find things to appreciate about The Host, but even die-hards will notice how disjointed things become. I can accept the half-hearted attempts at character development (at least one of which, wherein the characters doze off, should have been cut) and the sometimes cheesy special effects, but the movie's attempts at humor are forced and the political aspects of the storyline bloat the running length to the point where The Host threatens to wear out its welcome. The ending is so serious and so gloomy that it's like a slap in the face, although one could laud the movie for daring to go in this direction. It's unusual if a little depressing.

The Host is director Bong Joon-ho's third feature, and the one that will get the most widespread international distribution. It opened to great fanfare in South Korea, where it drew huge crowds and set box office records. (This leads to a question about whether there are cultural elements to a movie like this that cause it to play better in its country of origin than elsewhere.) The American reception is expected to be muted. Subtitles will curtail the size of potential audiences and the movie's uneven tone and pace will further erode box office intake. The Host is passable entertainment for monster movie lovers, but offers little of value to anyone else.
'''

"\nThe Host is a strange little movie: part creature feature, part social commentary, and part slapstick comedy. The problem with the film is that the sum isn't greater than the parts and the pieces don't fuse in a way that's consistently pleasing or cinematically satisfying. While the monster movie aspects of the film represent great fun, The Host's political subplot is overlong, uninteresting, and obvious. The comedy, which surfaces in seemingly random, broad bursts, is more often bizarre than funny (people taking pratfalls are the most common), and the ending is a downer.\n\nThe premise is simple enough: Chemicals dumped into the Han River from a U.S. military base create a mutant creature that grows alarmingly large. One day, the thing emerges from the water and starts eating passers-by. Hyun-seo (Ko Ah-sung), the pre-teen daughter of lazy Seoul snack vendor Gang-du (Song Kang-ho), is abducted by the monster and taken to its lair for future consumption. Gang-du mounts a rescue oper

In [50]:
rtxt="""
Gross-out horror is never far from comedy and The Host, Bong Joon-ho's giddy creature feature, has an anarchic mess factor worthy of a pile of old Mad magazines. A broadly played clown show full of lowbrow antics, Bong's big splat is itself a sort of monster—the top grossing movie in South Korean history—and, since it surfaced at Cannes last May, festival audiences having been slurping it down like ramen.
The Host's main attraction is a mutant carnivore-cum-somersaulting slimeball. As Mad once "animated" a garbage dump called the Heap, so The Host presents what might be a chunk of phlegm hawked from the maw of our despoiled earth. This killer tadpole can swim like a fish, scuttle like an insect, and run like a Spielberg raptor. Even more than the 1933 King Kong, Bong's creature is a surreal entity with no fixed size. As the materialization of dread, this nameless monster is harder to pin down than the radioactive, fire-breathing Godzilla. It's an "It."

Bong's allegory is deliberately free-floating; still, that the thing has its origins in American stupidity and hubris is made clear in The Host's prologue, set in a morgue on a U.S. Army base. Offended by the dust on some unused bottles of formaldehyde, an overbearing American officer orders a hapless Mr. Kim to dump gallons of toxic chemicals down the drain and into the Han River. A few years pass and two fishermen spot something gross swimming in the murk. . . . Cut to the wacky dysfunctional family who operate a riverside fast-food stand.

The Park clan consists of an elderly patriarch and his two deadbeat sons—one a slob, the other a drunk—and a daughter who is a championship archer with an unfortunate psychological hitch. There's also 11-year-old granddaughter Hyun-seo, courtesy of the slob—busy dishing out fried squid when he realizes that, down by the river, picnickers are transfixed by something suspended beneath the bridge.

The It falls into the water and swims over. Ordinary people, being what they are, merrily pelt the unknown creature with garbage until, with projectile force, it bounds ashore and the chase is on—thud, grab, leaping lizards! Establishing a galumphing tone of carnivalesque terror that trumps just about everything to follow, this picnic panic is a comic replay of 9/11 or even Sergei Eisenstein's "Odessa Steps." Then the thing dives back into the river, scattering a gaggle of swan-shaped paddleboats, with little Hyun-seo in its fishy clutches. From then on, it's personal.

Like the original, Japanese version of Godzilla, The Host gives catastrophe a naturalistic follow-through. A mass funeral for the monster's victims is held in a gymnasium that's housing traumatized survivors. The elder Park vows to rescue his granddaughter—or at least wreak vengeance. The girl's aunt solemnly offers up her bronze medal. Unlike the equivalent moments in Godzilla, however, this somber scene soon disintegrates into farce. The drunken brother arrives to immediately start blaming his siblings. The entire family is rolling-on-the-floor hysterical when the shelter is quarantined. The creature, it's explained—in such a way as to defy any rational explanation—was carrying a mysterious virus. But is it the It or South Korea who is really the host?

From the perspective of the Parks, the monster comes to embody whatever irrational forces oppress them. The authorities are essentially the It's agents; their main concern is subduing the "contaminated" family who, having received a cell phone call from Hyun-seo, are desperate to escape. Discovering that the creature is warehousing its victims, the Parks troll the roiling Han for their lost child. Meanwhile, the authorities are after the nonexistent virus. The sinister Americans are even planning to drill a guy's head for it: "The virus has definitely invaded his brain." It's what Borat called a "war of terror."

Bong, who has dealt with desperate pet-nappers and serial killers in his previous features—the crazy romantic comedy Barking Dogs Never Bite (2000) and the sociological policer Memories of Murder (2003)—has no difficulty integrating the horrifying, the stooge-like, and the everyday. (In that, he's even more extreme than our own masters of sociologic shock schlock—George Romero, Larry Cohen, and Joe Dante.) Just as grisly bio-horror is tricked out with cheesy effects and inappropriate music, so do spasms of naturalistic grief-coping alternate with pop-eyed slapstick. The Host is disgusting in some original and unforgettable ways, as when the monster vomits out human bones and an indigestible (or non-biodegradable) can of beer.

That can is key. Korea is imagined as someone's toxic waste dump. Criticized by the U.S. and the World Health Organization for bungling the situation, the authorities plan to spray Seoul with the evocatively named "Agent Yellow." (Not surprisingly, Bong is affiliated with South Korea's left-wing Democratic Labor Party.) The movie's climax conflates an anti-Agent Yellow protest, a police riot, and the family's inspirational last stand. Bong is a generous director. Although The Host has a tendency to repeat its routines, the filmmaker typically ends each scene with some offbeat comic lagniappe that serves to reground the fantasy in some quotidian morass.

As amorphous as its creature, The Host has an engaging refusal to take itself seri ously—it's no War of the Worlds and yet, however funny, it is hardly camp. The emotions that The Host churns up, regarding idiot authority and poisonous catastrophe, are too raw—too close to disgust. Is revulsion a form of revolt? Bong's disaster farce ends with a long shot of the frozen Han. There's the sense of something new brewing in the sludge—namely his movie.
"""

In [60]:
"Gross-out horror is never far from comedy and The Host, Bong Joon-ho\'s giddy creature feature, has an anarchic mess factor worthy of a pile of old Mad magazines. A broadly played clown show full of lowbrow antics, Bong\'s big splat is itself a sort of monster—the top grossing movie in South Korean history—and, since it surfaced at Cannes last May, festival audiences having been slurping it down like ramen.\nThe Host\'s main attraction is a mutant carnivore-cum-somersaulting slimeball. As Mad once \"animated\" a garbage dump called the Heap, so The Host presents what might be a chunk of phlegm hawked from the maw of our despoiled earth. This killer tadpole can swim like a fish, scuttle like an insect, and run like a Spielberg raptor. Even more than the 1933 King Kong, Bong\'s creature is a surreal entity with no fixed size. As the materialization of dread, this nameless monster is harder to pin down than the radioactive, fire-breathing Godzilla. It\'s an \"It.\"\n\nBong\'s allegory is deliberately free-floating; still, that the thing has its origins in American stupidity and hubris is made clear in The Host\'s prologue, set in a morgue on a U.S. Army base. Offended by the dust on some unused bottles of formaldehyde, an overbearing American officer orders a hapless Mr. Kim to dump gallons of toxic chemicals down the drain and into the Han River. A few years pass and two fishermen spot something gross swimming in the murk. . . . Cut to the wacky dysfunctional family who operate a riverside fast-food stand.\n\nThe Park clan consists of an elderly patriarch and his two deadbeat sons—one a slob, the other a drunk—and a daughter who is a championship archer with an unfortunate psychological hitch. There\'s also 11-year-old granddaughter Hyun-seo, courtesy of the slob—busy dishing out fried squid when he realizes that, down by the river, picnickers are transfixed by something suspended beneath the bridge.\n\nThe It falls into the water and swims over. Ordinary people, being what they are, merrily pelt the unknown creature with garbage until, with projectile force, it bounds ashore and the chase is on—thud, grab, leaping lizards! Establishing a galumphing tone of carnivalesque terror that trumps just about everything to follow, this picnic panic is a comic replay of 9/11 or even Sergei Eisenstein\'s \"Odessa Steps.\" Then the thing dives back into the river, scattering a gaggle of swan-shaped paddleboats, with little Hyun-seo in its fishy clutches. From then on, it\'s personal.\n\nLike the original, Japanese version of Godzilla, The Host gives catastrophe a naturalistic follow-through. A mass funeral for the monster\'s victims is held in a gymnasium that\'s housing traumatized survivors. The elder Park vows to rescue his granddaughter—or at least wreak vengeance. The girl\'s aunt solemnly offers up her bronze medal. Unlike the equivalent moments in Godzilla, however, this somber scene soon disintegrates into farce. The drunken brother arrives to immediately start blaming his siblings. The entire family is rolling-on-the-floor hysterical when the shelter is quarantined. The creature, it\'s explained—in such a way as to defy any rational explanation—was carrying a mysterious virus. But is it the It or South Korea who is really the host?\n\nFrom the perspective of the Parks, the monster comes to embody whatever irrational forces oppress them. The authorities are essentially the It\'s agents; their main concern is subduing the \"contaminated\" family who, having received a cell phone call from Hyun-seo, are desperate to escape. Discovering that the creature is warehousing its victims, the Parks troll the roiling Han for their lost child. Meanwhile, the authorities are after the nonexistent virus. The sinister Americans are even planning to drill a guy\'s head for it: \"The virus has definitely invaded his brain.\" It\'s what Borat called a \"war of terror.\"\n\nBong, who has dealt with desperate pet-nappers and serial killers in his previous features—the crazy romantic comedy Barking Dogs Never Bite (2000) and the sociological policer Memories of Murder (2003)—has no difficulty integrating the horrifying, the stooge-like, and the everyday. (In that, he\'s even more extreme than our own masters of sociologic shock schlock—George Romero, Larry Cohen, and Joe Dante.) Just as grisly bio-horror is tricked out with cheesy effects and inappropriate music, so do spasms of naturalistic grief-coping alternate with pop-eyed slapstick. The Host is disgusting in some original and unforgettable ways, as when the monster vomits out human bones and an indigestible (or non-biodegradable) can of beer.\n\nThat can is key. Korea is imagined as someone\'s toxic waste dump. Criticized by the U.S. and the World Health Organization for bungling the situation, the authorities plan to spray Seoul with the evocatively named \"Agent Yellow.\" (Not surprisingly, Bong is affiliated with South Korea\'s left-wing Democratic Labor Party.) The movie\'s climax conflates an anti-Agent Yellow protest, a police riot, and the family\'s inspirational last stand. Bong is a generous director. Although The Host has a tendency to repeat its routines, the filmmaker typically ends each scene with some offbeat comic lagniappe that serves to reground the fantasy in some quotidian morass.\n\nAs amorphous as its creature, The Host has an engaging refusal to take itself seri ously—it\'s no War of the Worlds and yet, however funny, it is hardly camp. The emotions that The Host churns up, regarding idiot authority and poisonous catastrophe, are too raw—too close to disgust. Is revulsion a form of revolt? Bong\'s disaster farce ends with a long shot of the frozen Han. There\'s the sense of something new brewing in the sludge—namely his movie."

'Gross-out horror is never far from comedy and The Host, Bong Joon-ho\'s giddy creature feature, has an anarchic mess factor worthy of a pile of old Mad magazines. A broadly played clown show full of lowbrow antics, Bong\'s big splat is itself a sort of monster—the top grossing movie in South Korean history—and, since it surfaced at Cannes last May, festival audiences having been slurping it down like ramen.\nThe Host\'s main attraction is a mutant carnivore-cum-somersaulting slimeball. As Mad once "animated" a garbage dump called the Heap, so The Host presents what might be a chunk of phlegm hawked from the maw of our despoiled earth. This killer tadpole can swim like a fish, scuttle like an insect, and run like a Spielberg raptor. Even more than the 1933 King Kong, Bong\'s creature is a surreal entity with no fixed size. As the materialization of dread, this nameless monster is harder to pin down than the radioactive, fire-breathing Godzilla. It\'s an "It."\n\nBong\'s allegory is del

In [58]:
"{}".format(rtxt.replace('"', '\\\"'))

'\nGross-out horror is never far from comedy and The Host, Bong Joon-ho\'s giddy creature feature, has an anarchic mess factor worthy of a pile of old Mad magazines. A broadly played clown show full of lowbrow antics, Bong\'s big splat is itself a sort of monster—the top grossing movie in South Korean history—and, since it surfaced at Cannes last May, festival audiences having been slurping it down like ramen.\nThe Host\'s main attraction is a mutant carnivore-cum-somersaulting slimeball. As Mad once \\"animated\\" a garbage dump called the Heap, so The Host presents what might be a chunk of phlegm hawked from the maw of our despoiled earth. This killer tadpole can swim like a fish, scuttle like an insect, and run like a Spielberg raptor. Even more than the 1933 King Kong, Bong\'s creature is a surreal entity with no fixed size. As the materialization of dread, this nameless monster is harder to pin down than the radioactive, fire-breathing Godzilla. It\'s an \\"It.\\"\n\nBong\'s alleg

In [52]:
rtxt

'\nGross-out horror is never far from comedy and The Host, Bong Joon-ho\'s giddy creature feature, has an anarchic mess factor worthy of a pile of old Mad magazines. A broadly played clown show full of lowbrow antics, Bong\'s big splat is itself a sort of monster—the top grossing movie in South Korean history—and, since it surfaced at Cannes last May, festival audiences having been slurping it down like ramen.\nThe Host\'s main attraction is a mutant carnivore-cum-somersaulting slimeball. As Mad once "animated" a garbage dump called the Heap, so The Host presents what might be a chunk of phlegm hawked from the maw of our despoiled earth. This killer tadpole can swim like a fish, scuttle like an insect, and run like a Spielberg raptor. Even more than the 1933 King Kong, Bong\'s creature is a surreal entity with no fixed size. As the materialization of dread, this nameless monster is harder to pin down than the radioactive, fire-breathing Godzilla. It\'s an "It."\n\nBong\'s allegory is d

In [75]:
'''
"The Host": It rose from American slime to destroy the Korean family! It must be destroyed!
Yes, the fishy, flippery, 60-foot-long thingummy who emerges from the polluted waters of the Han River in central Seoul to terrorize the populace in "The Host" is the result of poison from an American military facility. Well, what the hell else would cause such a horrible mutation? The fertilizer off Uncle Hang-soo's farm? I don't think so.

There's no question that Bong Joon-ho's film, which is the most satisfying monster movie in many years, takes some easy shots at the American military-technological colossus, and at the Korean government's sheepdog-like subservience to it. I'm inclined to interpret pretty much any junky old movie as a dialectical critique of whateverness, but in this case both the sanctimonious leftists and the contrarian critics are reading way too much into this simultaneously big-hearted and farcical adventure.

"The Host" may have one foot in the allegorical and mysterious world of contemporary Asian horror cinema, but the other one is closer to the sentimental big-screen spectacles of Steven Spielberg, or even Frank Capra. Bong's human villains are about as ambiguous as his monster: The Americans are diabolical Strangeloves and the Koreans are two-faced sycophants. His hero, on the other hand, is Gang-du (Song Kang-ho), a middle-aged loser with a bad blond 'do who slumbers away the days at his dad's riverside squid shack. (That isn't any kind of a joke: Koreans really, really like squid.)

Gang-du's dad (Byeon Hie-bong) is a grump and complainer, his beautiful sister (Bae Du-na) is a champion archer who loses an international match through indecision, and his brother (Park Hae-il) is an embittered alcoholic. About the only sensible one is Gang-du's daughter Hyun-seo (Ko Ah-sung), a resourceful little person of about 9 who puts up with this family as patiently as she can manage. Do you suppose coming face-to-face with a giant mutant whatzit will give this struggling family a chance at redemption, or what?

There's tragedy beneath the funniest bits of "The Host" and humor beneath the most serious passages, which is one reason why I don't think it should be viewed as some earnest political parable. When the big kahuna emerges from the muddy Han and begins gnawing up picnickers, the resulting chaos is both upsetting and comic -- at least until it grabs Hyun-seo and disappears. Even then, the film's tone is constitutionally unsettled: At the stage-managed ceremony of public grieving after the monster's first attack, the entire Park family collapses in a slapstick heap. As a stern orange-suited bureaucrat seeks to quarantine those who (like Gang-du) have actually touched the beast -- reputedly the host of a dangerous virus -- he wipes out in a pratfall worthy of Oliver Hardy.

Hyun-seo is gone, all right -- but is she really dead? After Gang-du gets a cellphone call from somewhere deep in the Han River sewer system, there is hope. Of course, as Gang-du explodes to a sinister American doctor late in the film, "Nobody ever fucking listens to me!" But our plucky family of working-class washouts doesn't give up easily. Infighting and bickering all the way, they take on the Korean cops and military, the evil and meddlesome Yanks (who want to release a poison gas called Agent Yellow in the city center), and, oh yeah, a hideous mutant creature who has their beloved Hyun-seo stashed somewhere for some future snacktime.

While the ignorance, hypocrisy and lies on display in "The Host" may result from America's increasingly clumsy quest to subjugate the earth in the name of freedom, those are the tools of authority in all historical eras and all nations. It's always those at the bottom of the food chain (those whom nobody ever fucking listens to!) who must battle the mutant monsters. On the margins of high-tech consumer society, the squid-shack proprietors will always be with us (as Jesus observed). "The Host" is a thrilling ride and a sometimes dry, sometimes sweet comedy, but beneath all that is a humane and tragic view of life worthy of the greatest films. Even those without rubber monsters.
'''

'\n"The Host": It rose from American slime to destroy the Korean family! It must be destroyed!\nYes, the fishy, flippery, 60-foot-long thingummy who emerges from the polluted waters of the Han River in central Seoul to terrorize the populace in "The Host" is the result of poison from an American military facility. Well, what the hell else would cause such a horrible mutation? The fertilizer off Uncle Hang-soo\'s farm? I don\'t think so.\n\nThere\'s no question that Bong Joon-ho\'s film, which is the most satisfying monster movie in many years, takes some easy shots at the American military-technological colossus, and at the Korean government\'s sheepdog-like subservience to it. I\'m inclined to interpret pretty much any junky old movie as a dialectical critique of whateverness, but in this case both the sanctimonious leftists and the contrarian critics are reading way too much into this simultaneously big-hearted and farcical adventure.\n\n"The Host" may have one foot in the allegorica

In [76]:
'''
The creature that rises out of a murky river to terrorize Seoul in Bong Joon-ho’s terrific The Host may put you in mind of a certain jowly shark, but its flapping jaws have a broader material and symbolic reach than any horror to emerge from the mind’s eye of Steven Spielberg. Bong, who won plaudits and awards around the world for his early movies Barking Dogs Never Bite and Memories of Murder, knows his way around the monster movie, but his predatory creation gives no quarter. Part frog, part fish and wholly a man-made nightmare, this beastly mutant doesn’t have a thing for lithe blond tootsies. It doesn’t make goo-goo eyes, lumber around murmuring “friends,” or pal around with soulfully brown-eyed little girls. Impartial and arbitrary, this equal-opportunity destroyer prowls the banks of the river Han, where the city’s poor gather to squeeze what little pleasure they can out of their downtrodden lives. Pouncing on old and young alike, it either flattens them on the spot or carries them off to a dank sewer, there to chew them right down to the bone.

I have next to no affinity for horror, but I left the theater charmed. Though there’s real terror in these bloody scenes, products of a fertile partnership between the special-effects labs that produced King Kong and Sin City, The Host is also a caper among whose many virtues is a complete absence of the knowing snicker that comes packaged with most contemporary creature features. At 37 years old, Bong still counts as a young Turk, but he brings to his subject a humanistic affection and expansive knowledge of the real world rarely found among filmmakers who single-mindedly nourished their childhood imaginations on things from black lagoons. The Host is a miracle of breathless play with form and tone that also seethes with attitude and ideas, from pure movie love to pointed sociopolitical commentary to a bleak existentialism about the inherent cruelty of our world. Like most movie monsters, Bong’s beast is Us — an expression of elemental fears, longings and anxieties about life and death in a universe run amok. But its random violence speaks to a peculiarly modern Us that heedlessly screws up our habitat, then rushes around trying in vain to contain the consequences.

The movie opens with the now obligatory act of American malice, in this case based on a real-life incident: A U.S. Army base officer (Scott Wilson, whose career has spanned the range of baleful Yanks) orders his reluctant Korean assistant to pour quantities of formaldehyde down a sink, knowing full well that it will make its way into the river. In due course, the chemical works its black magic and shows up as the bloated creature, whose most terrifying aspect is not its rather modest size as monsters go, but the mutant body parts — a split, gaping maw and scaly extra limbs — jutting from all the wrong places, and the lopsided lope with which it stalks its proletarian victims, among whom lounges an altogether more affable monster. Fat and lazy, his broad features framed by a tacky blond dye job, Park Gang-du (played with adorable zest by the popular South Korean actor Song Kang-ho) is as useless at running the riverside food stand he mans with his hard-working father as he is benignly neglectful of his daughter Hyun-seo (Ko A-sung). Yet this grinning slacker, apparently incapable of summoning up predatory energy if his life depended on it, shapes up into an unlikely hero when Hyun-seo’s life comes to depend on him and his wacky family, whose most potent weapon against the creature is a bow and arrow.

The monster may be the host of every current fear we have, from viruses to water contamination to global terror. But as Park searches for his missing daughter, he has more to fear from the pompous, bumbling officials who thwart him at every turn, and who, in an uproarious send-up of SARS hysteria, run around in silly yellow suits flapping their hands. Park redeems himself, but it’s not what you think, for though Bong is an unabashed populist (“Hot shots stink,” was his enchanting reply to an interviewer who asked why he’s drawn to working-class people), he’s no sentimentalist. Tucked inside the warmly hopeful ending of The Host, if you really pay attention, is a chilling conviction that, creature or no creature, life is an uphill struggle with limited success against an indifferent and unforgiving universe. Which makes you fear, as well it might, for the future of the one child who lives to console us in The Host, as the birth of another child did in Children of Men.

'''

'\nThe creature that rises out of a murky river to terrorize Seoul in Bong Joon-ho’s terrific The Host may put you in mind of a certain jowly shark, but its flapping jaws have a broader material and symbolic reach than any horror to emerge from the mind’s eye of Steven Spielberg. Bong, who won plaudits and awards around the world for his early movies Barking Dogs Never Bite and Memories of Murder, knows his way around the monster movie, but his predatory creation gives no quarter. Part frog, part fish and wholly a man-made nightmare, this beastly mutant doesn’t have a thing for lithe blond tootsies. It doesn’t make goo-goo eyes, lumber around murmuring “friends,” or pal around with soulfully brown-eyed little girls. Impartial and arbitrary, this equal-opportunity destroyer prowls the banks of the river Han, where the city’s poor gather to squeeze what little pleasure they can out of their downtrodden lives. Pouncing on old and young alike, it either flattens them on the spot or carries

In [80]:
'''
Bong Joon-ho's vastly entertaining creature feature The Host shattered box-office records in its native South Korea, which counts as an encouraging sign that Hollywood has lost its monopoly on effects-heavy escapism. But even that achievement sells short the film's specific virtues, like a daylight monster attack that could stand toe-to-toe with anything in Spielberg's oeuvre or the playful mix of tones that made Bong's previous film, Memories Of Murder, so distinctive. It can also be appreciated as a sweeping metaphor for America's toxic intervention abroad, though never to the point where it could be accused of high-mindedness. Most of all, The Host functions as a popcorn movie par excellence, loaded with the most familiar conventions, but shot through with such conviction and visual panache that even its clichés seem invigorating.

In the prologue, an American military officer orders dozens of dust-covered formaldehyde bottles to be poured down the drain, ignoring protests that the noxious liquid will go directly into the Han River, which flows into Seoul. Cut to six years later, when oafish layabout Song Kang-ho minds the riverside snack trailer owned by his family, including his scolding father Byeon Heui-bong, his young daughter Ko A-sung, and his sister Bae Du-na, who's in the national archery finals. (Not that her skills will figure into the story at some point down the line, of course.) Suddenly, on this ordinary afternoon, a giant creature with tentacles springs out of the river and starts devouring everyone in its path. When it scoops up Ko and deposits her in its sewer lair, it's up to Song and his family to rescue the girl before it's too late.

Bong and his special-effects team take a major risk by exposing their creature to the light of day, where it could easily look cheesy or disconnected from the real environment, but the CGI could pass muster in a production with 10 times its budget. Still, what really sells the monster is Bong's impressive arsenal of old-fashioned visual tricks, like tucking crucial information in the corner of the screen or outside the frame altogether. At a full two hours, The Host could use a little trimming, especially in the sagging middle section between the first-rate opening action and slapstick sequences and the suitably gripping climactic showdown. But Bong still offers a powerful rejoinder to his Hollywood counterparts: Anything you can do, I can do better.
'''

"\nBong Joon-ho's vastly entertaining creature feature The Host shattered box-office records in its native South Korea, which counts as an encouraging sign that Hollywood has lost its monopoly on effects-heavy escapism. But even that achievement sells short the film's specific virtues, like a daylight monster attack that could stand toe-to-toe with anything in Spielberg's oeuvre or the playful mix of tones that made Bong's previous film, Memories Of Murder, so distinctive. It can also be appreciated as a sweeping metaphor for America's toxic intervention abroad, though never to the point where it could be accused of high-mindedness. Most of all, The Host functions as a popcorn movie par excellence, loaded with the most familiar conventions, but shot through with such conviction and visual panache that even its clichés seem invigorating.\n\nIn the prologue, an American military officer orders dozens of dust-covered formaldehyde bottles to be poured down the drain, ignoring protests that

In [81]:
'''
Like all good hosts, the host in "The Host," a mutant squid-lizard that moves with the agility of an Olympic gymnast, throws a lively party with a little of everything: scares, laughs, politics and a bit of archery.

The South Korean writer-director Bong Joon-ho has made a considerable international splash with this picture, and no wonder. It boasts a photogenic antagonist from the deep. It's also savvy enough to make you care about the human factor. Like "Pan's Labyrinth," another (but very different) R-rated fantasy in which a young girl confronts evil beyond her imagining, "The Host" takes its perils seriously.

Fulfilling one expectation of its genre, Bong's film is about the fracturing and reassembling of an extended family. On the banks of Seoul's Han River, snack shop proprietor Kang-du (Song Kang-ho) loses his middle-school-age daughter (Ko A-sung) to the slimy tentacles of the massive squid-lizard, in the monster's first major assault. The scene is beautifully paced; from the first sighting of the creature, in droll, matter-of-fact medium shot, you know you're in assured directorial hands.

To Kang-du's father, brother and archer-sister, the girl's abduction-she and a young boy are kept alive in the creature's lair underneath a city bridge-is merely the latest proof that Kang-du is hapless. Dad has a score to settle with his own reputation. "The Host" sticks close to one family's response to the crisis, while taking shots at the reasons for the trouble. In a prologue, a U.S. Army officer, in charge of the morgue in Seoul, orders an underling to dump toxic chemicals down the drain. (This is based on a real-life incident, though as far as we know a monster-free one.) Years later, bingo: squid-zard in the river, thanks to American arrogance.

South Korea doesn't come off much better. In cahoots with the Americans, the government sprays witnesses to the first attack with something called "Agent Yellow." Later, in a throwaway line, we learn that post-attack cleanup operations have been farmed out to 14 different private companies, none of which seem to know what it's doing. The tone of "The Host" is slippery in the best way; you're never sure if you're in for a joke or a shock, yet nothing feels random. It's on the longish side, and it has a couple of extra endings, but the film (as William H. Macy said of Joe Mantegna in "Homicide") is like an aquarium. There's something going on every minute.

Director Bong's previous feature, "Memories of Murder" (2003), was a slippery fish as well, treating the pursuit of a serial killer with a sardonic touch, just this side of facetious. He's a wit, and he knows how to deploy wit (as opposed to gags, though he's not above a few choice sight gags in "The Host") to further a narrative. At the same time Bong knows what a chosen genre demands. When someone in "The Host" edges close to the water and says, "It's an Amazonian river dolphin for sure!" you know full well he's about to be proven wrong.

'''

'\nLike all good hosts, the host in "The Host," a mutant squid-lizard that moves with the agility of an Olympic gymnast, throws a lively party with a little of everything: scares, laughs, politics and a bit of archery.\n\nThe South Korean writer-director Bong Joon-ho has made a considerable international splash with this picture, and no wonder. It boasts a photogenic antagonist from the deep. It\'s also savvy enough to make you care about the human factor. Like "Pan\'s Labyrinth," another (but very different) R-rated fantasy in which a young girl confronts evil beyond her imagining, "The Host" takes its perils seriously.\n\nFulfilling one expectation of its genre, Bong\'s film is about the fracturing and reassembling of an extended family. On the banks of Seoul\'s Han River, snack shop proprietor Kang-du (Song Kang-ho) loses his middle-school-age daughter (Ko A-sung) to the slimy tentacles of the massive squid-lizard, in the monster\'s first major assault. The scene is beautifully pace

In [85]:
'''
Hey, kids, you wanna see the biggest blockbuster in the history of Korean cinema? No? How about the out-and-out scariest monster movie to come down the pike since, oh, Aliens?

There, I thought so. The monster who pops out of the Han river to terrorize Seoul in Bong Joon-ho's wildly entertaining The Host sure is an elaborately ugly thing (imagine if the Alien creature concept was based on a prawn), but how it looks isn't nearly as jump-out-of-your-seat startling as how and when it shows up. And the out-and-out scariest bits in the film aren't when the creature is attacking, but when one of its lead characters — most particularly the plucky little girl heroine, Park Hyun-seo (Ko A-sung), who's been kidnapped and stowed away by the creature — is waiting for it to attack.

When the movie isn't being scary, it's crazily funny, so much so that critical watchers will wonder if Bong might tilt the balance of the picture too far in a comic direction and water down the scares. He doesn't. Instead, he gets both laughs and cheers for Park Hyun-seo's ridiculously dysfunctional clan, including her super-dense dad (Song Kang-ho, who played a somewhat similarly deficient detective in Bong's excellent 2003 thriller Memories of Murder) and crossbow-wielding aunt. Their story is forefronted as all of Seoul goes into a panic, and the Korean authorities are shunted aside by U.S. military personnel (who, a prologue shows, are responsible for the creature's creation in the first place), who then screw things up even further. I understand that some people are particularly sensitive about other countries giving us a hard time in their movies (I always figured the U.S.A. was tough enough to take it); still, the America-bashing in this picture is done in such a blandly matter-of-fact fashion that it registers as much funnier than it might have.

But the movie is hardly a full-fledged satire; part of what makes it so enjoyable is how much stuff it contains while never losing sight of its prime directive, which is to thrill and terrify. The movie's finale is dauntingly different from what a Hollywood version of this picture might have offered. Leaving my first viewing of The Host, I thought I would have really loved the movie even if its ending had copped out.
'''

"\nHey, kids, you wanna see the biggest blockbuster in the history of Korean cinema? No? How about the out-and-out scariest monster movie to come down the pike since, oh, Aliens?\n\nThere, I thought so. The monster who pops out of the Han river to terrorize Seoul in Bong Joon-ho's wildly entertaining The Host sure is an elaborately ugly thing (imagine if the Alien creature concept was based on a prawn), but how it looks isn't nearly as jump-out-of-your-seat startling as how and when it shows up. And the out-and-out scariest bits in the film aren't when the creature is attacking, but when one of its lead characters — most particularly the plucky little girl heroine, Park Hyun-seo (Ko A-sung), who's been kidnapped and stowed away by the creature — is waiting for it to attack.\n\nWhen the movie isn't being scary, it's crazily funny, so much so that critical watchers will wonder if Bong might tilt the balance of the picture too far in a comic direction and water down the scares. He doesn't

In [86]:
'''
It's big and ugly, and before it started terrorizing innocent Koreans, it was a tadpole. Then a careless American doctor at a local military base ordered hundreds of gallons of formaldehyde dumped into the Han River. The creature swallowed the toxin; now the thing is 30 ft. long, has 10 legs, looks like an angry Muppet and is itching for mischief. U.S. scientists have yet more dire news: the beast is the host for a deadly virus that could wipe out everyone in Seoul or-- dare we say it--the world!

The Host is itself a bizarre hybrid: both a popular hit (South Korea's all-time box-office champ) and a critics' choice, having played to acclaim at the Cannes and New York festivals. The film's plot is also pretty splitty: part old-time sea- or sewer-mutant movie in the tradition of Godzilla and Them! and part trigenerational comedy-drama about a weird family--sort of a Little Miss Korean Sunshine. The difference is that instead of a dead old man in the van, the Park family has a little girl (Ko A-sung) missing in the belly of the beast. Then director Bong Joon-ho sets one more plate spinning, with easy-to-spot political metaphors for U.S. influence in Korea and sleight-of-hand in Iraq. If this madly entertaining movie has a fault, it's that it's too ingenious for the genre it ostensibly inhabits.

The poor Park family seems farcically overmatched by the monster. The sharpshooter dad runs out of bullets; the girl archery expert can't shoot straight. Actually, you won't want them to kill the monster, not right away, since it has lots of its own eccentricities. The creature is less vicious than playful, a showboating athlete that does high-bar 360s on a bridge rail and backflips into the river. When it hits land, it lopes like Marmaduke next to its ostensible victims; it treats any human in its mouth more as a chew toy than as lunch. If the movie is remade for the U.S. market, expect kids to beg for those monster toys.

But Hollywood is sure to sand and sanitize this film's quirks. See the real thing now, not a ghost of The Host later.
'''

"\nIt's big and ugly, and before it started terrorizing innocent Koreans, it was a tadpole. Then a careless American doctor at a local military base ordered hundreds of gallons of formaldehyde dumped into the Han River. The creature swallowed the toxin; now the thing is 30 ft. long, has 10 legs, looks like an angry Muppet and is itching for mischief. U.S. scientists have yet more dire news: the beast is the host for a deadly virus that could wipe out everyone in Seoul or-- dare we say it--the world!\n\nThe Host is itself a bizarre hybrid: both a popular hit (South Korea's all-time box-office champ) and a critics' choice, having played to acclaim at the Cannes and New York festivals. The film's plot is also pretty splitty: part old-time sea- or sewer-mutant movie in the tradition of Godzilla and Them! and part trigenerational comedy-drama about a weird family--sort of a Little Miss Korean Sunshine. The difference is that instead of a dead old man in the van, the Park family has a little

In [87]:
'''
How come we get so many films about serial killers, teen-age libidos, and the Second World War, but nothing about giant mutant tadpoles? The imbalance has always struck me as unjust, and some of us have considered forming a pressure group to lobby for the inclusion of giant mutant tadpoles in mainstream cinema. Now the pressure is off, thanks to “The Host.” The director is Bong Joon-ho, and the mainstream in question is the Han, which flows through the center of Seoul. The rapid economic advance of South Korea in the nineteen-sixties and seventies is commonly known in the region as “the Miracle on the Han.” Here comes the downturn.

Song Kangho and Ko Asung flee the monster in a new film from South Korea.
Song Kang-ho and Ko A-sung flee the monster in a new film from South Korea.R. KIKUO JOHNSON
In the opening scene, a load of old formaldehyde, tipped down the drain by a lab technician, ends up in the waters of the Han. A few years later, we see the result, though we can’t quite make out what we are looking at: a dark, bulbous, comma-shaped form, drooping from the struts of a bridge. Slowly, it unfurls, like an Olympic diver, and drops neatly into the water. The good people of Seoul, amused by such novelty, toss beer cans at it and take snapshots on their cell phones. One of them glances sideways, at which point amusement comes to a halt—unlike the beast, which has hopped onto the embankment and is now pounding toward the crowd, plainly in search of a snack to go with the beer. Reading from back to front, we find: a lashing tail, which also functions as a lasso; legs as thick as a dinosaur’s, with claws that can churn up concrete; eyes as dull as glue; and a mouth that seems to peel open in many directions at once. The whole package is the size of a Greyhound bus, and extremely vexed about something. It looks like Broderick Crawford crossed with a Venus flytrap.

Observing the monster is Gang-du (Song Kang-ho), a slob who runs a nearby food stand. He has yellow hair, the ability to fall asleep at any time, and—somewhat to our surprise—a young daughter, Hyun-seo (Ko A-sung). She gets caught up in the crowd’s chaotic flight from the monster, which promptly kidnaps her and, in obedience to the rules of fairy tale, bears her off to its lair. The rest of the film charts the efforts of Gang-du, aided by his family, to retrieve his only child from the dripping, bone-strewn sewer where she now resides. The story drags slightly in the middle, but there is no mistaking the rampant acceleration of the final act, in which all those present are called upon to save a life and, in so doing, to fulfill the task for which each of them is essentially equipped. For example, the hero’s sister, Nam-joo (Bae Doo-na), is a professional archer—always handy for those pesky dragonslaying situations that crop up so often these days. Her other brother, Nam-il (Park Hae-il), spends half the movie complaining that, as a college graduate, he can’t get a job, whereupon, at the climax, he turns into an angry student—preparing Molotov cocktails, like a soixante-huitard, and lobbing them at the leathery flanks of his foe.

No one can claim that we live in a golden age—or a bile-green, suppurating age—of movie monsters, and there will be plenty of filmgoers who yawned through “Godzilla” in 1998 and swore off large amphibians for good. All I can say, to tempt them back, is that I have seen “The Host” twice and have every intention of watching it again. Anyone who has taken the plunge into recent Korean movies (“Lady Vengeance” and “Oldboy,” say) will know that their impact springs not just from the verve of the storytelling but from a tendency to hurtle energetically from one mood to the next, merrily swapping the lyrical for the sadistic. It is as if Korean directors, refusing to observe the niceties of genre, offered value for money by packing several movies into one. The trick with “The Host,” the most successful release in the history of Korean cinema, is therefore to unpack it and inspect the contents.

VIDEO FROM THE NEW YORKER

What Went Wrong in the Sewol Ferry Disaster?


The first component is shock, without which no monster movie can hope to thrive. “The Host” delivers a fine series of jolts, as the beast, which uses the industrial architecture of Seoul as an acrobat uses the parallel bars, swings suddenly out of the darkness. More unorthodox is the visual gag near the beginning, when Gang-du, fleeing the rampage, takes hold of his daughter’s hand, then loses it in the rush. The screen slows and turns quiet; his hand gropes for hers and finds it again, at which point we pull back to reveal that he has grabbed the wrong girl. This is typical of Bong Joon-ho: the setting may be outlandish, but the emotion—every parent’s fear—could hardly be more rooted in the real.

Second, there is the yuck factor. “The Host” is not especially bloody, and it will hold limited appeal for those macho viewers who like to test and prove themselves by cackling through a film like “Hostel,” which was little better than a dramatized abattoir. Nonetheless, the special effects have an insidious slither of their own. They were created by the Orphanage, the California company that worked on “Hellboy,” “Sin City,” and recent installments of Harry Potter and “Pirates of the Caribbean.” What the wizards have conjured for “The Host,” though, takes me back to the heyday of John Carpenter—specifically, to the flowering nastiness of “The Thing.” The emphasis is not on weaponry (the best defense turns out to be a metal pole) but on the organic, as befits a beast that was spawned by environmental pollution—watch it regurgitate victims like a mother disgorging fish to its young in “March of the Penguins.” At one point, Hyun-seo leaps into the air and stays there. For a few seconds, we don’t get it. (Bong likes to prolong the spell.) Then we switch angles, and all becomes clear: the girl is held aloft in the coils of the monster’s sticky tail.

Third, Bong, like Carpenter, treats authority with the scorn of the prankster at the back of the class. Anyone in a uniform is fair game. Some guy in a yellow protective suit, seeking to inform the public of what has occurred on the Han, soon gives up and suggests weakly that people watch it on TV. When the girl’s family is held in a hospital, following rumors that the beast may be the source—the host—of a menacing virus, we get a delicate probing of the generational split. Gang-du’s father is all for mollifying the doctors, trying to slip them a sweetener in cash, whereas the son wants to muscle his way out as fast as possible. This niggle of disrespect lends the film a nervous litheness; we want the monster quelled, and order restored, but what, exactly, is so peaceful about the status quo?

All of which leads to the director’s fourth and most unlikely virtue: he makes you laugh. Moreover, he keeps doing so in the wrong—or, at any rate, the least laughable—places. This was demonstrated in his earlier movie “Memories of Murder” (2003), in which detectives and forensic experts arriving to study a corpse kept slipping on an embankment and landing on their behinds, like extras in a Mack Sennett comedy. Many graceful directors—Jacques Tati, obviously, but also Krzysztof Kieslowski—have been drawn to the sheer gracelessness of humanity, as if there were something endearing in our habit of falling (or tumbling) short of that noble dignity which, when sitting comfortably, we like to ascribe to ourselves. The most destabilizing scene in “The Host” comes as Gang-du and the others wail with grief before a photograph of Hyun-seo, whom they believe to be dead, and make such a spectacle of themselves, writhing like eels, that despite ourselves we snicker at their collapse. Such maladroit behavior is unusual in a monster flick, where you expect at least a modicum of cool; you can’t really imagine Sigourney Weaver, at the climax of “Aliens,” tripping over her own flamethrower.

Given all that, it may seem perverse to hail “The Host” as a thing of beauty, yet that is what it is: a perfect mixture of the silly and the grave. In 1954, the art critic David Sylvester, impatient with the accepted canon, wrote a review of “Them!,” in Encounter, in which he described it as “a frightening, imaginative, beautifully written B-movie-style SF film about giant ants, exactly the sort of film the professional critics could be guaranteed not to latch on to.” Time to make amends. We have shifted from ants to tadpoles (“The Host” could easily have been titled “It!”), but the style is intact. Take the gorgeous wide shot of the Han River, early in the film, with rain gusting across the surface and a hint of something more than murky stirring in its depths. Then leapfrog to the last scene: we are back at Gang-du’s food stand, after dark, with adult and child sharing a heaped table of food. On the television, there is news about the beast—a report, voiced by American officials, about the unseemly events in Seoul. The feasters in the shack, who, after all, were principal players in that drama, want nothing more to do with it: “Should we watch something else?” “Let’s turn it off. Concentrate on eating.” The whole flustered panoply of TV, camera crews, and cell phones that has flashed and clicked throughout the movie is turned off, making way for a glimpse of an older, more peaceable Korea. We move outside for a closing image worthy of Fellini: the small hut, so light and festive within, warmed by the steam of rice, is marooned in a landscape of night and drifting snow. Anything could be out there.

'''

'\nHow come we get so many films about serial killers, teen-age libidos, and the Second World War, but nothing about giant mutant tadpoles? The imbalance has always struck me as unjust, and some of us have considered forming a pressure group to lobby for the inclusion of giant mutant tadpoles in mainstream cinema. Now the pressure is off, thanks to “The Host.” The director is Bong Joon-ho, and the mainstream in question is the Han, which flows through the center of Seoul. The rapid economic advance of South Korea in the nineteen-sixties and seventies is commonly known in the region as “the Miracle on the Han.” Here comes the downturn.\n\nSong Kangho and Ko Asung flee the monster in a new film from South Korea.\nSong Kang-ho and Ko A-sung flee the monster in a new film from South Korea.R. KIKUO JOHNSON\nIn the opening scene, a load of old formaldehyde, tipped down the drain by a lab technician, ends up in the waters of the Han. A few years later, we see the result, though we can’t quite

In [88]:
'''
Within the first fifteen minutes of “Gwoemul,” director Joon-Ho manages to build up enough tension and characterization to charge at us head on with the central plot. Based on true events, one day an American scientist orders his assistant to empty the remaining jars of Chloroform into his sink. His assistant, in his own feeble way, attempts to protest, but he immediately gives in and dumps almost a hundred jars into the Han River. “Gwoemul” is not just a monster movie, it’s a movie about family, and it’s a movie about the lengths we’ll go to to save our loved ones.
The fantastic yet horrifying attack of the sea monster at the Han River is not only one of the most amazing action scenes to ever be filmed, but Joon-Ho then sideswipes the audience by painting the aftermath of the attack as a true tragedy. In a gut wrenching scene, hundreds grieve their loved ones eaten by the monster, and the victims stand at an altar filled with pictures of the victims as the mourners cry hysterically.

The family we center on is grieving the death of their youngest Hyun-Seo who was taken by the creature and dragged away into the water. To further twist the conventions of the plot, the beast has now infected its victims with a mysterious virus that has spread along their skin, and soon they realize that the beast is only part of the horrors coming to them once the government takes control.

Infected with--what the local government calls a virus--the family breaks free from the hospital to save Hyun-seo who is trapped in a sewer. Joon-Ho not only explores the ravages of the monster, but the chain reaction this massive attack causes on the government, the community, and this disconnected family who decide to band together to save one of their own. One a mentally incapable man, one an Olympic archer, a feeble old man, and an alcoholic.

With very strong performances from the entire cast, and rich character focus, Joon-Ho plays their relationship with startling emotion, leaving them wholly outmatched, but powered by their relentless drive to save the youngest of the family. Joon-Ho’s tale of environmental chaos, government induced hysteria, and a monster that is only instrumental in the bigger picture of anarchy, and the collapse of a civilization under man.

But apocalypse be damned, they’ll confront military, scientists, mobs of head hunters, and an unstoppable beast to get their family back, and that’s enough for them. Joon-Ho’s epic is a masterpiece of monster cinema that’s intelligent, innovative, and reaches down to the basic core of family unity to propel its story beyond mere conventions of science fiction.

“Gwoemul” has been a financial success in its country, praised by many film fans, critically acclaimed, and sports one of the most dynamic stories I’ve seen in years. How could the US not remake it? But they’ll do it all wrong, because in the end, “Gwoemul” is not a monster movie. It’s about a monster that comes between a family.
'''

'\nWithin the first fifteen minutes of “Gwoemul,” director Joon-Ho manages to build up enough tension and characterization to charge at us head on with the central plot. Based on true events, one day an American scientist orders his assistant to empty the remaining jars of Chloroform into his sink. His assistant, in his own feeble way, attempts to protest, but he immediately gives in and dumps almost a hundred jars into the Han River. “Gwoemul” is not just a monster movie, it’s a movie about family, and it’s a movie about the lengths we’ll go to to save our loved ones.\nThe fantastic yet horrifying attack of the sea monster at the Han River is not only one of the most amazing action scenes to ever be filmed, but Joon-Ho then sideswipes the audience by painting the aftermath of the attack as a true tragedy. In a gut wrenching scene, hundreds grieve their loved ones eaten by the monster, and the victims stand at an altar filled with pictures of the victims as the mourners cry hystericall

In [89]:
'''
"The Host" isn't just a terrific monster movie. This South Korean box-office smash is also a laugh-out-loud comedy and a surprisingly angry political satire.

It starts when the U.S. Army dumps hundreds of gallons of toxic formaldehyde in Seoul's Han River. (The officer in charge wants to tidy up the lab, you see.)

Flash-forward six years: A giant walking trout-monster -- a literal land shark -- leaps out of the river in broad daylight . . . and starts munching on Korean consumers.

 Advertisement
	


	


The monster was designed by Weta Workshop, which did design work on "The Lord of the Rings" trilogy. As directed by Bong Joon-ho, the creature attack is one of the best ever staged -- a ballet of mass panic with pulsing music matching the monster's rhythmic gait. It's brilliant, terrifying and weirdly funny.

It also leaves you wondering: Why was the monster fully revealed in the first 10 minutes of the movie?

The answer: Because it's a prelude to the real horror, as "The Host" turns into a satire of government misinformation and corruption that plays like "Godzilla" crossed with "Dr. Strangelove."

After the attack, a dim-bulb single dad named Park Gang-du (Song Kang-ho) gets a cell phone call from his 13-year-old daughter. The girl was dragged underwater by the monster and presumed dead -- but she's actually hiding in the beast's sewer lair.

It's joyful news: The monster is stupid, the call can be traced and the army's already on the scene. But Park can't get government bureaucrats to believe him and mount a rescue.

"She's deceased, but she's not dead!" Park screams. A flunky replies that a cell phone trace is "not something we do for just anyone." Instead, officials get hung up on the idea that the monster spreads a virus -- and do little more than detain anyone they think might be infected.

Soon, Park enlists his bickering family of losers in a vigilante hunt for the monster. The movie does an incredible job juggling laughs, suspense and a deep-seated distrust of bureaucracy.

"The Host" is wildly entertaining; people who love the genre will get their geek buttons pushed hard by this flick. But what makes the movie an instant classic is that it's also, in its dark heart, a furious lampoon. I love how the creature constantly shows up after the appearance of consumer products like beer and packaged snacks. One of the funniest and most biting scenes comes early, when an impromptu grief memorial dissolves into pratfalls and wailing, all captured by a huddle of television cameras.

While the South Korean and U.S. governments initially are lampooned for laughs, it gets less and less funny as the story becomes more and more tragic. "The Host," in its own silly monster-movie way, is actually kind of a call to arms -- demanding that people wake up, take responsibility for their lifestyles and (above all) ask tough questions of their government and mass media. Failure to do so is more destructive than any land shark.
'''

'\n"The Host" isn\'t just a terrific monster movie. This South Korean box-office smash is also a laugh-out-loud comedy and a surprisingly angry political satire.\n\nIt starts when the U.S. Army dumps hundreds of gallons of toxic formaldehyde in Seoul\'s Han River. (The officer in charge wants to tidy up the lab, you see.)\n\nFlash-forward six years: A giant walking trout-monster -- a literal land shark -- leaps out of the river in broad daylight . . . and starts munching on Korean consumers.\n\n Advertisement\n\t\n\n\n\t\n\n\nThe monster was designed by Weta Workshop, which did design work on "The Lord of the Rings" trilogy. As directed by Bong Joon-ho, the creature attack is one of the best ever staged -- a ballet of mass panic with pulsing music matching the monster\'s rhythmic gait. It\'s brilliant, terrifying and weirdly funny.\n\nIt also leaves you wondering: Why was the monster fully revealed in the first 10 minutes of the movie?\n\nThe answer: Because it\'s a prelude to the real

So it looks like I can get some of these reviews, but again, I was not able to get all 35. Therefore, I will explore Nexis Uni to see if I can rely on that instead of Metacritic or Rotten Tomatoes for critic reviews.

On Nexis Uni, I can search for reviews of both movies that were posted in newspaper, blog, or magazine articles, so I may be able to get the full document text and go through an easier process for analyzing the critic reviews.

For this, please refer to `nexis_host_critic` and `nexis_parasite_critic`, and to look at the analysis of critic reviews, please refer to `analysis_critic_nexis`. Thanks!